# Part 4: Using GPU acceleration with PyTorch

In [0]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

## Manual use of `.cuda()`

Now the magic of PyTorch comes in. So far, we've only been using the CPU to do computation. When we want to scale to a bigger problem, that won't be feasible for very long.
|
PyTorch makes it really easy to use the GPU for accelerating computation. Consider the following code that computes the element-wise product of two large matrices:

In [0]:
import torch

t1 = torch.randn(1000, 1000)
t2 = torch.randn(1000, 1000)
t3 = t1*t2
print(t3)

tensor([[ 1.2057,  1.3576, -0.2003,  ..., -0.0436,  0.1404, -2.3319],
        [ 1.0632,  0.9236,  0.0092,  ..., -0.9863, -0.2660,  5.9091],
        [ 0.1943,  1.2426, -0.7572,  ...,  0.3998, -0.0631,  1.1385],
        ...,
        [ 0.0706,  0.8910,  0.0170,  ...,  0.1098,  1.0510, -0.9269],
        [-0.0481,  2.2200,  4.9920,  ..., -4.8261,  2.7089, -0.9079],
        [-1.3935,  0.6256, -0.9238,  ...,  0.1120,  0.6053,  3.2615]])


By sending all the tensors that we are using to the GPU, all the operations on them will also run on the GPU without having to change anything else. If you're running a non-cuda enabled version of PyTorch the following will throw an error; if you have cuda available the following will create the input matrices, copy them to the GPU and perform the multiplication on the GPU itself:

In [0]:
t1 = torch.randn(1000, 1000).cuda()
t2 = torch.randn(1000, 1000).cuda()
t3 = t1*t2
print(t3)

tensor([[ 0.1168,  0.1375,  0.2083,  ..., -0.3258, -0.0704, -2.5908],
        [-0.6055,  0.3875, -0.2094,  ..., -0.5521,  1.1159, -0.7061],
        [-0.5628,  1.0472,  0.7277,  ..., -0.2489, -0.8462, -0.1377],
        ...,
        [ 0.0921,  0.0679, -0.9423,  ...,  0.5658,  0.0594,  0.1212],
        [-0.2795,  0.1344, -0.4582,  ..., -0.1827,  0.3040,  0.0289],
        [-0.5796, -0.0457, -0.7919,  ...,  0.1024,  0.3811,  0.3249]],
       device='cuda:0')


If you're running this workbook in colab, now enable GPU acceleration (`Runtime->Runtime Type` and add a `GPU` in the hardware accelerator pull-down). You'll then need to re-run all cells to this point.

If you were able to run the above with hardware acceleration, the print-out of the result tensor would show that it was an instance of `cuda.FloatTensor` type on the the `(GPU 0)` GPU device. If your wanted to copy the tensor back to the CPU, you would use the `.cpu()` method.

## Writing platform agnostic code

Most of the time you'd like to write code that is device agnostic; that is it will run on a GPU if one is available, and otherwise it would fall back to the CPU. The recommended way to do this is as follows:

In [0]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t1 = torch.randn(1000, 1000).to(device)
t2 = torch.randn(1000, 1000).to(device)
t3 = t1*t2
print(t3)

tensor([[-0.0576, -1.2623,  0.4279,  ..., -0.5060,  0.0099, -2.7000],
        [-1.3462, -0.0046,  0.8698,  ..., -0.3178, -0.0168,  0.0296],
        [-0.6702, -0.3287, -0.3749,  ..., -0.1019, -0.9066, -0.2658],
        ...,
        [-0.3101,  0.5127,  0.3136,  ...,  0.0458,  0.5199,  0.4513],
        [-0.0572, -0.4512,  0.8361,  ...,  0.0519,  0.3161,  0.0145],
        [-0.0467, -0.1121,  0.6163,  ..., -0.1351, -0.5228, -0.2547]],
       device='cuda:0')


## Accelerating neural net training

If you wanted to accelerate the training of a neural net using raw PyTorch, you would have to copy both the model and the training data to the GPU. Unless you were using a really small dataset like MNIST, you would typically _stream_ the batches of training data to the GPU as you used them in the training loop:

```python
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = BaselineModel(784, 784, 10).to(device)

loss_function = ...
optimiser = ...

for epoch in range(10):
    for data in trainloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimiser.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()
```

Using Torchbearer, this becomes much simpler - you just tell the `Trial` to run on the GPU and that's it!:

```python
model = BetterCNN()

loss_function = ...
optimiser = ...

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader)
trial.run(epochs=10)
```


## Multiple GPUs

Using multiple GPUs is beyond the scope of the lab, but if you have multiple cuda devices, they can be referred to by index: `cuda:0`, `cuda:1`, `cuda:2`, etc. You have to be careful not to mix operations on different devices, and would need how to carefully orchestrate moving of data between the devices (which can really slow down your code to the point at which using the CPU would actually be faster).

## Questions

__Answer the following questions (enter the answer in the box below each one):__

__1.__ What features of GPUs allow them to perform computations faster than a typically CPU?


GPU's are able to do a lot of parallel computations. A Lot more than a CPU can do.

__2.__ What is the biggest limiting factor for training large models with current generation GPUs?

GPU memory limitation